In [27]:
import requests
import json

# api-endpoint 

URL = "http://neu-3-1:18080/api/v1/applications"


# sending get request and saving the response as response object 
r = requests.get(url = URL) 
  
# extracting data in json format 
applications = r.json() 

In [28]:
import time 
import datetime 

def get_stages(app_id):
    print(app_id)
    URL = "http://neu-3-1:18080/api/v1/applications/%s/stages"%(app_id)
    # sending get request and saving the response as response object 
    if URL == "http://neu-3-1:18080/api/v1/applications/app-20200617160405-0036/stages":
        print(URL)
        
    URL="http://neu-3-1:18080/api/v1/applications/app-20200617160405-0036/stages"
        
    r = requests.get(url = URL) 
    #print(r)
    # extracting data in json format 
    stages = r.json() 

    for s in stages:
        submit_time = time.mktime(datetime.datetime.strptime(s['submissionTime'], 
                                                     "%Y-%m-%dT%H:%M:%S.%fGMT").timetuple())        
        start_time = time.mktime(datetime.datetime.strptime(s['firstTaskLaunchedTime'], 
                                                     "%Y-%m-%dT%H:%M:%S.%fGMT").timetuple())
        completion_time = time.mktime(datetime.datetime.strptime(s['completionTime'], 
                                                     "%Y-%m-%dT%H:%M:%S.%fGMT").timetuple())
        #print(s['stageId'], s['inputBytes'], s['outputBytes'], s['rddIds'],
        #      s['numTasks'], completion_time - submit_time)
        
        #print(s.keys())
        #break
        
for app in applications:
    if app['name'] == 'name:wordcount-bw:40Gbps-ds:1G-stride:0-rep:0-jcxywltu':
        get_stages(app['id'])
        break
    
    
r

<Response [200]>

In [29]:
import pandas as pd

df = pd.read_csv('/local0/Kariz/expriments/runtime_prediction/micro/results/wordcount-spark.csv')
df

,app_id,app_name,stage_id,input_sz,output_sz,rdds,n_tasks,runtime,name,bw,dataset,stride,rep
0,app-20200617165846-0043,name:wordcount-bw:40Gbps-ds:1G-stride:0-rep:0-...,40Gbps,1G,0,64,wordcount,26655,"[6, 5]",0,7.0,1,0
1,app-20200617165846-0043,name:wordcount-bw:40Gbps-ds:1G-stride:0-rep:0-...,40Gbps,1G,1102238868,8,wordcount,0,"[4, 2, 0, 3, 1]",0,8.0,0,0
2,app-20200617175135-0044,name:wordcount-bw:40Gbps-ds:1G-stride:0-rep:0-...,40Gbps,1G,0,64,wordcount,26655,"[6, 5]",0,8.0,1,0
3,app-20200617175135-0044,name:wordcount-bw:40Gbps-ds:1G-stride:0-rep:0-...,40Gbps,1G,1102238868,8,wordcount,0,"[4, 2, 3, 1, 0]",0,9.0,0,0
4,app-20200617175613-0046,name:wordcount-bw:40Gbps-ds:1G-stride:0-rep:0-...,40Gbps,1G,0,64,wordcount,26655,"[6, 5]",0,10.0,1,0
5,app-20200617175613-0046,name:wordcount-bw:40Gbps-ds:1G-stride:0-rep:0-...,40Gbps,1G,1102238868,8,wordcount,0,"[4, 3, 1, 0, 2]",0,9.0,0,0
6,app-20200617175857-0048,name:wordcount-bw:40Gbps-ds:1G-stride:0-rep:0-...,40Gbps,1G,0,64,wordcount,26655,"[6, 5]",0,7.0,1,0
7,app-20200617175857-0048,name:wordcount-bw:40Gbps-ds:1G-stride:0-rep:0-...,40Gbps,1G,1102238868,8,wordcount,0,"[4, 2, 0, 3, 1]",0,10.0,0,0
8,app-20200617180249-0051,name:wordcount-bw:40Gbps-ds:1G-stride:0-rep:0-...,40Gbps,1G,0,64,wordcount,26655,"[6, 5]",0,8.0,1,0
9,app-20200617180249-0051,name:wordcount-bw:40Gbps-ds:1G-stride:0-rep:0-...,40Gbps,1G,1102238868,8,wordcount,0,"[4, 1, 0, 3, 2]",0,10.0,0,0


In [100]:
import re


class Stage():
    def __init__(self, indent):
        self.id = -1
        self.parents = {}
        self.children = {}
        self.indent = indent
        self.lines = []

stages = []

def _parse_rdd_string(rdd_str):
    lines = rdd_str.split('\n')
    
    n_stages = 0
    prev_indent = -1
    cur_stage = None
    
    for ln in lines:
        if '+-' in n:
            s = Stage(ln.index('+-'))
            s.stage_id = len(stages)
            stages.append(s)
            
            if cur_stage and cur_stage.indent < s.indent:
                cur_stage.parents[s.id] = cur_stage
            elif cur_stage and cur_stage.indent == s.indent:
                pass
            print(ln, ln.index('+-'))
            n_stages += 1
    print(n_stages)

def parse_rdd_string(rdd_str):
    lines = rdd_str.split('\n')
    
    rdd_str = '''+-(1) MapPartitionsRDD[58] at save at TpchQuery.scala:42 []
 |  ShuffledRowRDD[57] at save at TpchQuery.scala:42 []
 +-(200) MapPartitionsRDD[56] at save at TpchQuery.scala:42 []
     |   MapPartitionsRDD[55] at save at TpchQuery.scala:42 []
     |   MapPartitionsRDD[54] at save at TpchQuery.scala:42 []
     |   MapPartitionsRDD[53] at save at TpchQuery.scala:42 []
     |   ZippedPartitionsRDD2[52] at save at TpchQuery.scala:42 []
     |   MapPartitionsRDD[46] at save at TpchQuery.scala:42 []
     |   ShuffledRowRDD[45] at save at TpchQuery.scala:42 []
     +-(200) MapPartitionsRDD[44] at save at TpchQuery.scala:42 []
         |   MapPartitionsRDD[43] at save at TpchQuery.scala:42 []
         |   ZippedPartitionsRDD2[42] at save at TpchQuery.scala:42 []
         |   MapPartitionsRDD[36] at save at TpchQuery.scala:42 []
         |   ShuffledRowRDD[35] at save at TpchQuery.scala:42 []
         +-(128) MapPartitionsRDD[34] at save at TpchQuery.scala:42 []
             |   MapPartitionsRDD[33] at save at TpchQuery.scala:42 []
             |   MapPartitionsRDD[32] at save at TpchQuery.scala:42 []
             |   MapPartitionsRDD[3] at map at TpchSchemaProvider.scala:90 []
             |   MapPartitionsRDD[2] at map at TpchSchemaProvider.scala:90 []
             |   /64G/customer/customer.tbl* MapPartitionsRDD[1] at textFile at TpchSchemaProvider.scala:90 []
             |   /64G/customer/customer.tbl* HadoopRDD[0] at textFile at TpchSchemaProvider.scala:90 []
         |   MapPartitionsRDD[41] at save at TpchQuery.scala:42 []
         |   ShuffledRowRDD[40] at save at TpchQuery.scala:42 []
         +-(128) MapPartitionsRDD[39] at save at TpchQuery.scala:42 []
             |   MapPartitionsRDD[38] at save at TpchQuery.scala:42 []
             |   MapPartitionsRDD[37] at save at TpchQuery.scala:42 []
             |   MapPartitionsRDD[19] at map at TpchSchemaProvider.scala:102 []
             |   MapPartitionsRDD[18] at map at TpchSchemaProvider.scala:102 []
             |   /64G/orders/orders.tbl* MapPartitionsRDD[17] at textFile at TpchSchemaProvider.scala:102 []
             |   /64G/orders/orders.tbl* HadoopRDD[16] at textFile at TpchSchemaProvider.scala:102 []
     |   MapPartitionsRDD[51] at save at TpchQuery.scala:42 []
     |   ShuffledRowRDD[50] at save at TpchQuery.scala:42 []
     +-(384) MapPartitionsRDD[49] at save at TpchQuery.scala:42 []
         |   MapPartitionsRDD[48] at save at TpchQuery.scala:42 []
         |   MapPartitionsRDD[47] at save at TpchQuery.scala:42 []
         |   MapPartitionsRDD[7] at map at TpchSchemaProvider.scala:93 []
         |   MapPartitionsRDD[6] at map at TpchSchemaProvider.scala:93 []
         |   /64G/lineitem/lineitem.tbl* MapPartitionsRDD[5] at textFile at TpchSchemaProvider.scala:93 []
         |   /64G/lineitem/lineitem.tbl* HadoopRDD[4] at textFile at TpchSchemaProvider.scala:93 []'''
    
    n_stages = 0
    s = None
    for idx, ln in enumerate(lines):
        if '+-' in ln:
            s = Stage(ln.index('+-'))
            s.stage_id = len(stages)
            stages.append(s)
            lines[idx] = lines[idx] + " " + str(ln.index('+-'))
        if s:
            s.lines.append(ln)
    lns = []
    for ln in lines:
        if '+-' in ln:
            splits = ln.split(':')
            name = 'MapPartitionsRDD[' + splits[0].split('MapPartitionsRDD')[1].split(']')[0].split('[')[1] + ']'
            val = splits[1]
            lns.append({'name': name, 'value': val, 'level': tab_level(ln)})
            n_stages += 1
    
    print(n_stages)
    return lns

def build_dag(lines):
    lns = []
    

def ttree_to_json(ttree,level=0):
    result = {}
    for i in range(0,len(ttree)):
        cn = ttree[i]
        try:
            nn  = ttree[i+1]
        except:
            nn = {'level':-1}

        # Edge cases
        if cn['level']>level:
            continue
        if cn['level']<level:
            return result

        # Recursion
        if nn['level']==level:
            dict_insert_or_append(result,cn['name'],cn['value'])
        elif nn['level']>level:
            rr = ttree_to_json(ttree[i+1:], level=nn['level'])
            dict_insert_or_append(result,cn['name'],rr)
        else:
            dict_insert_or_append(result,cn['name'],cn['value'])
            return result
    return result
    
    
def ttree_to_json(ttree,level=0):
    result = {}
    for i in range(0,len(ttree)):
        cn = ttree[i]
        try:
            nn  = ttree[i+1]
        except:
            nn = {'level':-1}

        # Edge cases
        if cn['level']>level:
            continue
        if cn['level']<level:
            return result

        # Recursion
        if nn['level']==level:
            dict_insert_or_append(result,cn['name'],cn['value'])
        elif nn['level']>level:
            rr = ttree_to_json(ttree[i+1:], level=nn['level'])
            dict_insert_or_append(result,cn['name'],rr)
        else:
            dict_insert_or_append(result,cn['name'],cn['value'])
            return result
    return result    
    
def dict_insert_or_append(adict,key,val):
    """Insert a value in dict at key if one does not exist
    Otherwise, convert value to list and append
    """
    if key in adict:
        if type(adict[key]) != list:
            adict[key] = [adict[key]]
        adict[key].append(val)
    else:
        adict[key] = val
    
def tab_level(astr):
    """Count number of leading tabs in a string
    """
    ret = len(astr)- len(astr.lstrip(' '))
    
    return (ret - 1)//4 + 1 if ret > 1 else ret 
    
lines = parse_rdd_string(rdd_str)
ret = ttree_to_json(lines)

print(json.dumps(ret, indent=10))

6
{
          "MapPartitionsRDD[58]": {
                    "MapPartitionsRDD[56]": {
                              "MapPartitionsRDD[44]": {
                                        "MapPartitionsRDD[34]": "42 [] 9",
                                        "MapPartitionsRDD[39]": "42 [] 9"
                              },
                              "MapPartitionsRDD[49]": "42 [] 5"
                    }
          }
}


In [101]:
import graph_tool.all as gt

def build_graph(g, ret):
    print(g)
    for node_name in ret:
        v = g.add_vertex()
        g.vp.name[v] = node_name 
    
g = gt.Graph(directed=True)
build_graph(g, ret)